# CamHD Video File to Azure Blob with Dask

In [ ]:
## inspect your database
import pandas as pd
# this option will show the full filename
pd.set_option('display.max_colwidth', -1)
dbcamhd = pd.read_json('/home/jovyan/floc_data/dbcamhd.json', orient="records", lines=True).sort_values(by=['timestamp'])
#dbcamhd.tail()

In [ ]:
# lets make a small set to test with
c = []
url = []
for i, row in dbcamhd[14400:14407].iterrows():
    file = row.filename
    c.append(file.split('/')[-1])
    url.append(file)

In [ ]:
# put the data in a dataframe
short = pd.DataFrame({'file': c, 'urls': url})

In [ ]:
# short

#### Start a Dask cluster

In [ ]:
#get yer dask going
from dask_kubernetes import KubeCluster
cluster = KubeCluster(n_workers=10)
cluster

In [ ]:
from dask.distributed import Client
import dask
import requests
import os
import s
from dask import delayed
import dask.array as dsa
client = Client(cluster)
client

## Dask Dl Function

In [ ]:
blob_names = short['file']
address = short['urls']

In [ ]:
print(len(blob_names))
print(len(address))

In [ ]:
#Trying dict comprehensions, because dataframes and lists are writing all downloads to the same file path... writing over the same file.
copy_dict = dict(zip(address, blob_names))

In [ ]:
# # import modules
import azure.storage.blob as ASB
import azure.storage.common
import yaml

In [ ]:
# load Azure storage account credentials
with open('/home/jovyan/.azure_credentials_ooitest.yaml', 'r') as f:
    credentials = yaml.load(f)
azure_storage_account_name = credentials['azure_storage_account_name']
azure_storage_account_key = credentials['azure_storage_account_key']

In [ ]:
block_blob_service = ASB.BlockBlobService(azure_storage_account_name, azure_storage_account_key)
container_name ='batchtesttwo'
block_blob_service.create_container(container_name);

In [ ]:
#Here we will try to DL with Dask
### the print command did not work but I would like it to!

@delayed
def delayed_copy_file(copy_dict):
    ## we use k and v here for key and value from our dictionary (copy_dict)
    for address,blob_names in copy_dict.items():
        # NOTE the stream=True parameter
        r = block_blob_service.copy_blob(container_name, blob_names, address)
        delayed_copy = delayed(r)
    delayed_copy_file = dsa.stack(delayed_copy)
    delayed_copy_file
    print("File" , blob_names , "created in", container_name)

In [ ]:
delayed_copy_file(copy_dict)

In [ ]:
%%time
results = delayed_copy_file.compute()

In [ ]:
delayed_frame_list = []
for i, row in scene_windows[scene_windows.deployment == 5].iterrows():
    filename = row.url
    delayed_moov_atom = delayed(camhd.get_moov_atom)(filename)
    for frame_number in row.frame_list:
        delayed_frame = delayed(camhd.get_frame)(filename, frame_number, 'gray16le', delayed_moov_atom)
        delayed_frame_list.append(dsa.from_delayed(delayed_frame, (1080, 1920), np.uint16))
delayed_frame_array = dsa.stack(delayed_frame_list)
delayed_frame_array

## Upload file from URL to Microsoft Azure Blob Storage

In [ ]:
from dask import delayed, compute

@delayed
def delayed_mov_to_blob(blob_service, container_name, blob_name, url):
    block_blob_service.copy_blob(container_name, blob_name, url)
    return 0

#### Set list of delayed functions

In [ ]:
delayed_copy = []
for i, row in short.iterrows():
    url = row.urls
    blob_name = row.file
    delayed_copy.append(delayed_mov_to_blob(block_blob_service, container_name, blob_name, url))

In [ ]:
delayed_copy[0]


In [ ]:
%%time
copy_across = compute(*delayed_copy)

In [ ]:
# define function to copy from remote url to blob storage
def run_sample():
       block_blob_service.copy_blob(container_name,'rawdata','https://rawdata.oceanobservatories.org/files/RS03ASHS/PN03B/06-CAMHDA301/2019/07/09/CAMHDA301-20190709T031500.mov')



In [ ]:
 # run function
if __name__ == '__main__':
     run_sample()

## Check size of containters & blobs to see if it worked

In [ ]:
containers = block_blob_service.list_containers()
for c in containers:
        contName = c.name
        generator = block_blob_service.list_blobs(contName)
        contSize = 0
        contNum = 0
        for blob in generator: 
            contSize += blob.properties.content_length
            contNum += 1
        print(contName + " : " + str(contSize/1000000) + " MB" + " in " + str(contNum) + " blobs")

## Clean up your space before you go home

### clean up blobs

In [ ]:
blob_list = block_blob_service.list_blobs(container_name)
for blob in blob_list: 
    blob_name = blob.name
    print(blob_name)

In [ ]:
# # delete all blobs in container
# blob = block_blob_service.list_blobs(container_name)
# #for a in blob:
#        #blob_service.delete_blob(container_name = container_name, blob_name = a.name)

### clean up containers

In [ ]:
# # #set container to delete
# container_name = 'mp4test'

In [ ]:
# # Clean up resources. This includes the container and the temp files.
# block_blob_service.delete_container(container_name)
# # os.remove(full_path_to_file)
# # os.remove(full_path_to_file2)